<a href="https://colab.research.google.com/github/mingyeom121212/aipython/blob/main/optimizing_BHF_with_MSELoss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('DATA2.csv')

# 특성과 타겟 분리
X = data[['BHF1', 'BHF2', 'BHF3', 'BHF4', 'BHF5']].values
y = data[['W2', 'W4', 'Crack']].values

# 데이터 표준화
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# 손실 함수 및 최적화 방법 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    loss = criterion(outputs, y_test)
    print(f'Test Loss: {loss.item():.4f}')

# 최적의 BHF 값을 찾기 위해 최적화
BHF_initial = torch.tensor(scaler_X.transform([[0, 0, 0, 0, 0]]), dtype=torch.float32, requires_grad=True)
optimizer = optim.Adam([BHF_initial], lr=0.01)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(BHF_initial)
    loss = criterion(outputs, torch.tensor([[0, 0, 0]], dtype=torch.float32))  # 목표 W2, W4, Crack 값
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Optimization Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

optimal_BHF = scaler_X.inverse_transform(BHF_initial.detach().numpy())
print(f'Optimal BHF values: {optimal_BHF}')


Epoch [100/1000], Loss: 0.0667
Epoch [200/1000], Loss: 0.0345
Epoch [300/1000], Loss: 0.0220
Epoch [400/1000], Loss: 0.0171
Epoch [500/1000], Loss: 0.0142
Epoch [600/1000], Loss: 0.0111
Epoch [700/1000], Loss: 0.0069
Epoch [800/1000], Loss: 0.0044
Epoch [900/1000], Loss: 0.0037
Epoch [1000/1000], Loss: 0.0031
Test Loss: 0.9060
Optimization Epoch [100/1000], Loss: 0.0575
Optimization Epoch [200/1000], Loss: 0.0429
Optimization Epoch [300/1000], Loss: 0.0303
Optimization Epoch [400/1000], Loss: 0.0200
Optimization Epoch [500/1000], Loss: 0.0123
Optimization Epoch [600/1000], Loss: 0.0071
Optimization Epoch [700/1000], Loss: 0.0039
Optimization Epoch [800/1000], Loss: 0.0020
Optimization Epoch [900/1000], Loss: 0.0009
Optimization Epoch [1000/1000], Loss: 0.0004
Optimal BHF values: [[ -5390.9795  37027.727  334302.25    38703.37    85877.09  ]]


In [ ]:
# 최적화 단계에서 음수가 발생하여 최적화 단계에서 BHF값이 음수가 되지 않도록 Clamping 하였음

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('DATA2.csv')

# 특성과 타겟 분리
X = data[['BHF1', 'BHF2', 'BHF3', 'BHF4', 'BHF5']].values
y = data[['W2', 'W4', 'Crack']].values

# 데이터 표준화
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# 손실 함수 및 최적화 방법 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    loss = criterion(outputs, y_test)
    print(f'Test Loss: {loss.item():.4f}')

# 최적의 BHF 값을 찾기 위해 최적화
BHF_initial = torch.tensor(scaler_X.transform([[0, 0, 0, 0, 0]]), dtype=torch.float32, requires_grad=True)
optimizer = optim.Adam([BHF_initial], lr=0.01)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(BHF_initial)
    loss = criterion(outputs, torch.tensor([[0, 0, 0]], dtype=torch.float32))  # 목표 W2, W4, Crack 값
    loss.backward()
    optimizer.step()

    # BHF 값을 음수가 되지 않도록 클램핑
    with torch.no_grad():
        BHF_initial.clamp_(min=0.0)

    if (epoch+1) % 100 == 0:
        print(f'Optimization Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

optimal_BHF = scaler_X.inverse_transform(BHF_initial.detach().numpy())
print(f'Optimal BHF values: {optimal_BHF}')


Epoch [100/1000], Loss: 0.0751
Epoch [200/1000], Loss: 0.0396
Epoch [300/1000], Loss: 0.0214
Epoch [400/1000], Loss: 0.0180
Epoch [500/1000], Loss: 0.0165
Epoch [600/1000], Loss: 0.0157
Epoch [700/1000], Loss: 0.0151
Epoch [800/1000], Loss: 0.0144
Epoch [900/1000], Loss: 0.0133
Epoch [1000/1000], Loss: 0.0111
Test Loss: 0.3189
Optimization Epoch [100/1000], Loss: 0.0001
Optimization Epoch [200/1000], Loss: 0.0001
Optimization Epoch [300/1000], Loss: 0.0001
Optimization Epoch [400/1000], Loss: 0.0001
Optimization Epoch [500/1000], Loss: 0.0001
Optimization Epoch [600/1000], Loss: 0.0001
Optimization Epoch [700/1000], Loss: 0.0001
Optimization Epoch [800/1000], Loss: 0.0001
Optimization Epoch [900/1000], Loss: 0.0001
Optimization Epoch [1000/1000], Loss: 0.0001
Optimal BHF values: [[ 24857.37   34418.395 468782.84  119186.42  232086.62 ]]


In [ ]:
#위에서는 W2,W4,Crack에 대한 BHF값들을 한번에 최적화 하였다면 아래의 코드는
#각각의 W2,W4,Crack에 대한 BHF값들을 최적화 하고 그 값들의 평균을 내었음

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 로드
data = pd.read_csv('DATA2.csv')

# 특성과 타겟 분리
X = data[['BHF1', 'BHF2', 'BHF3', 'BHF4', 'BHF5']].values
y = data[['W2', 'W4', 'Crack']].values

# 데이터 표준화
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# 손실 함수 및 최적화 방법 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    loss = criterion(outputs, y_test)
    print(f'Test Loss: {loss.item():.4f}')

# 최적의 BHF 값을 찾기 위한 함수
def find_optimal_BHF(target_index):
    BHF_initial = torch.tensor(scaler_X.transform([[0, 0, 0, 0, 0]]), dtype=torch.float32, requires_grad=True)
    optimizer = optim.Adam([BHF_initial], lr=0.01)
    target_tensor = torch.zeros((1, 3), dtype=torch.float32)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(BHF_initial)
        loss = criterion(outputs[0, target_index], target_tensor[0, target_index])
        loss.backward()
        optimizer.step()

        # BHF 값을 음수가 되지 않도록 클램핑
        with torch.no_grad():
            BHF_initial.clamp_(min=0.0)

        if (epoch+1) % 100 == 0:
            print(f'Optimization for target {target_index} Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    return BHF_initial.detach().numpy()

# W2, W4, Crack 각각에 대해 최적화된 BHF 값을 구함
optimal_BHF_W2 = find_optimal_BHF(0)
optimal_BHF_W4 = find_optimal_BHF(1)
optimal_BHF_Crack = find_optimal_BHF(2)

# 최적화된 BHF 값의 평균 계산
optimal_BHF_avg = (optimal_BHF_W2 + optimal_BHF_W4 + optimal_BHF_Crack) / 3
optimal_BHF_avg = scaler_X.inverse_transform(optimal_BHF_avg)
print(f'Optimal BHF values (average): {optimal_BHF_avg}')


Epoch [100/1000], Loss: 0.0733
Epoch [200/1000], Loss: 0.0334
Epoch [300/1000], Loss: 0.0206
Epoch [400/1000], Loss: 0.0148
Epoch [500/1000], Loss: 0.0082
Epoch [600/1000], Loss: 0.0051
Epoch [700/1000], Loss: 0.0038
Epoch [800/1000], Loss: 0.0035
Epoch [900/1000], Loss: 0.0032
Epoch [1000/1000], Loss: 0.0031
Test Loss: 11.9652
Optimization for target 0 Epoch [100/1000], Loss: 0.0008
Optimization for target 0 Epoch [200/1000], Loss: 0.0000
Optimization for target 0 Epoch [300/1000], Loss: 0.0000
Optimization for target 0 Epoch [400/1000], Loss: 0.0000
Optimization for target 0 Epoch [500/1000], Loss: 0.0000
Optimization for target 0 Epoch [600/1000], Loss: 0.0000
Optimization for target 0 Epoch [700/1000], Loss: 0.0000
Optimization for target 0 Epoch [800/1000], Loss: 0.0000
Optimization for target 0 Epoch [900/1000], Loss: 0.0000
Optimization for target 0 Epoch [1000/1000], Loss: 0.0000
Optimization for target 1 Epoch [100/1000], Loss: 0.0314
Optimization for target 1 Epoch [200/1000]